In [1]:
import pandas as pd
import numpy as np
import torch
import datetime
from resample import random_oversample
import itertools
import os

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print(f"Training on {DEVICE} using PyTorch {torch.__version__}")
print('main1_mlp_federated')

Training on cpu using PyTorch 1.13.1+cpu
main1_mlp_federated


In [3]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('./dataset/data.csv').astype('float32')
X = df.drop('label', axis=1)
df['label'] = df['label'].replace(21, 19)
y = df['label']

X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.int).reshape(-1,1)

In [4]:
# Define train and test function
from sklearn.metrics import mean_squared_error, roc_auc_score, f1_score

def prediction2label(pred: np.ndarray):
    """Convert ordinal predictions to class labels, e.g.

    [0.9, 0.1, 0.1, 0.1] -> 0
    [0.9, 0.9, 0.1, 0.1] -> 1
    [0.9, 0.9, 0.9, 0.1] -> 2
    etc.
    """
    return (pred > 0.5).cumprod(axis=1).sum(axis=1) - 1

def test(net, x_test, y_test):
    y_pred = net(x_test)
    if y_pred.shape[1] == 20:
        y_pred = prediction2label(net(x_test))
    else:
        y_pred = torch.clip(y_pred.round(), min=0, max=20)

    y_default_test = np.where(y_test.detach().numpy() > 9, 1, 0)
    y_default_pred = np.where(y_pred.detach().numpy() > 9, 1, 0)
    mse = mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy())
    mse_norm = mean_squared_error(y_test.detach().numpy() / 21, y_pred.detach().numpy() / 21)
    auc = roc_auc_score(y_default_test, y_default_pred)
    f1 = f1_score(y_default_test, y_default_pred)
    return mse, mse_norm, auc, f1


In [5]:
# Define MLP FL simulation wrapper
from fl_simu import mlp_simulation

def run_simulation(split, X_train, X_test, y_train, y_test, dir='default', num_epoch=256, batch_size=16, num_rounds=15, history_dir='./history/example.txt'):
    return mlp_simulation(split, X_train, y_train, X_test, y_test, dir, num_epoch, batch_size, num_rounds, history_dir=history_dir)


D:\FD\venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
random_state = np.random.randint(1000)

from sklearn.model_selection import StratifiedKFold

batch_size_list = [8]
num_epochs_list = [4]
num_rounds_list = [60] # 40

def do_fl(splitting, X=X, y=y):

    start = datetime.datetime.now()
    mses = []
    aucs = []

    test_name = 'Dataset 1 MLP'
    directory_path = 'model_checkpoints/' + test_name

    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    print(type(X))
    print(type(y))

    flag = 1

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        print(f"Fold {fold_idx + 1}:")

        if flag == 2:
            break

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        for batch_size, num_epoch, num_rounds in itertools.product(batch_size_list, num_epochs_list, num_rounds_list):
            X_test = X_test.to(DEVICE)
            y_test = y_test.to(DEVICE)

            filename = 'mlp_batch_'+ str(batch_size) + '_epochs_' + str(num_epoch) + '_rounds_' + str(num_rounds) + '_loss.csv'
            history_dir = './history/dataset1/' + filename

            model_fl = run_simulation(splitting, X_train, X_test, y_train, y_test, test_name, batch_size=batch_size, num_epoch=num_epoch, num_rounds=num_rounds, history_dir=history_dir)

            model_fl.cpu()

            X_test = X_test.cpu()
            y_test = y_test.cpu()

            mse, mse_norm, auc, f1 = test(model_fl, X_test, y_test)
            mses.append(mse)
            aucs.append(auc)

            flag += 1

    end = datetime.datetime.now()
    time = end - start

    print('split', splitting)
    print('time', time)
    print('mse', mses)
    print('auc', aucs)


In [7]:
split = [0.5, 0.5]
do_fl(split)

INFO flwr 2023-08-31 01:59:19,158 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([2800, 22])
Client 1 trainset shape torch.Size([2820, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 01:59:26,675	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 01:59:34,950 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 524294555.0, 'object_store_memory': 262147276.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 01:59:34,953 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 01:59:34,955 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 01:59:39,870 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 01:59:39,871 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 01:59:39,872 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 01:59:39,873 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 01:59:49,856 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 01:59:49,864 | fedavg.p

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  3.0406654343807764


DEBUG flwr 2023-08-31 01:59:56,393 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 01:59:56,415 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 01:59:56,456 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 01:59:56,458 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 2)


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  1.7634011090573012


DEBUG flwr 2023-08-31 02:06:38,188 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:06:38,230 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:06:38,275 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:06:38,278 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  1.8077634011090573


DEBUG flwr 2023-08-31 02:06:45,134 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:06:45,152 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:06:45,195 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:06:45,196 | server.py:153 | FL finished in 425.3230099999928
INFO flwr 2023-08-31 02:06:45,197 | app.py:225 | app_fit: losses_distributed [(1, 3.0406654343807764), (2, 3.142329020332717), (3, 2.765249537892791), (4, 2.467652495378928), (5, 2.401109057301294), (6, 2.3622920517560075), (7, 2.700554528650647), (8, 2.5397412199630316), (9, 2.5841035120147873), (10, 2.7153419593345656), (11, 2.4861367837338264), (12, 2.6432532347504623), (13, 2.1460258780036967), (14, 2.4417744916820703), (15, 2.410351201478743), (16, 2.048059149722736), (17, 2.33456561922366), (18, 2.3937153419593344), (19, 2.325323475046211), (20, 2.412199630314233), (21, 2.086876155268022

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  1.7264325323475047
History (loss, distributed):
	round 1: 3.0406654343807764
	round 2: 3.142329020332717
	round 3: 2.765249537892791
	round 4: 2.467652495378928
	round 5: 2.401109057301294
	round 6: 2.3622920517560075
	round 7: 2.700554528650647
	round 8: 2.5397412199630316
	round 9: 2.5841035120147873
	round 10: 2.7153419593345656
	round 11: 2.4861367837338264
	round 12: 2.6432532347504623
	round 13: 2.1460258780036967
	round 14: 2.4417744916820703
	round 15: 2.410351201478743
	round 16: 2.048059149722736
	round 17: 2.33456561922366
	round 18: 2.3937153419593344
	round 19: 2.325323475046211
	round 20: 2.412199630314233
	round 21: 2.086876155268022
	round 22: 1.8428835489833642
	round 23: 2.2513863216266174
	round 24: 1.9057301293900184
	round 25: 2.0646950092421443
	round 26: 1.9630314232902033
	round 27: 2.081330868761553
	round 28

In [8]:
split = [0.6, 0.4]
do_fl(split)

INFO flwr 2023-08-31 02:06:45,321 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([3460, 22])
Client 1 trainset shape torch.Size([1944, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:06:54,147	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:07:01,736 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 905564160.0, 'memory': 1811128320.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:07:01,738 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:07:01,741 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:07:05,218 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:07:05,219 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:07:05,221 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:07:05,221 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:07:14,916 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:07:14,924 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  4.844731977818854


DEBUG flwr 2023-08-31 02:07:22,042 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:07:22,065 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:07:22,147 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:07:22,148 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 2)


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  1.866913123844732


DEBUG flwr 2023-08-31 02:15:25,071 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:15:25,087 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:15:25,118 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:15:25,119 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  2.0979667282809613


DEBUG flwr 2023-08-31 02:15:32,514 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:15:32,534 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:15:32,566 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:15:32,567 | server.py:153 | FL finished in 507.34610580001026
INFO flwr 2023-08-31 02:15:32,568 | app.py:225 | app_fit: losses_distributed [(1, 4.844731977818854), (2, 2.7634011090573014), (3, 2.6303142329020335), (4, 2.866913123844732), (5, 2.536044362292052), (6, 2.7837338262476896), (7, 2.4842883548983363), (8, 2.3992606284658042), (9, 2.2051756007393717), (10, 2.4805914972273566), (11, 2.3475046210720887), (12, 2.412199630314233), (13, 2.765249537892791), (14, 2.5175600739371533), (15, 2.1072088724584104), (16, 2.1275415896487986), (17, 2.2033271719038816), (18, 2.011090573012939), (19, 2.290203327171904), (20, 2.166358595194085), (21, 2.46395563770

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  1.9630314232902033
History (loss, distributed):
	round 1: 4.844731977818854
	round 2: 2.7634011090573014
	round 3: 2.6303142329020335
	round 4: 2.866913123844732
	round 5: 2.536044362292052
	round 6: 2.7837338262476896
	round 7: 2.4842883548983363
	round 8: 2.3992606284658042
	round 9: 2.2051756007393717
	round 10: 2.4805914972273566
	round 11: 2.3475046210720887
	round 12: 2.412199630314233
	round 13: 2.765249537892791
	round 14: 2.5175600739371533
	round 15: 2.1072088724584104
	round 16: 2.1275415896487986
	round 17: 2.2033271719038816
	round 18: 2.011090573012939
	round 19: 2.290203327171904
	round 20: 2.166358595194085
	round 21: 2.463955637707948
	round 22: 2.2144177449168208
	round 23: 2.1090573012939
	round 24: 2.0036968576709797
	round 25: 2.44362292051756
	round 26: 2.3401109057301293
	round 27: 2.229205175600739
	round 28: 

In [9]:
split = [0.8, 0.2]
do_fl(split)

INFO flwr 2023-08-31 02:15:32,674 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([4580, 22])
Client 1 trainset shape torch.Size([988, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:15:42,093	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:15:49,727 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 787821772.0, 'memory': 1575643547.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:15:49,730 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:15:49,732 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:15:53,451 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:15:53,452 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:15:53,453 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:15:53,453 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:16:03,391 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:16:03,400 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  2.5138632162661736


DEBUG flwr 2023-08-31 02:16:12,894 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:16:12,914 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:16:12,955 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:16:12,956 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 2)


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  1.8817005545286507


DEBUG flwr 2023-08-31 02:25:39,697 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:25:39,725 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:25:39,761 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:25:39,762 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 2)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  1.878003696857671


DEBUG flwr 2023-08-31 02:25:49,493 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:25:49,509 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-08-31 02:25:49,541 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:25:49,542 | server.py:153 | FL finished in 596.0880122000235
INFO flwr 2023-08-31 02:25:49,543 | app.py:225 | app_fit: losses_distributed [(1, 2.5138632162661736), (2, 2.622920517560074), (3, 2.532347504621072), (4, 2.9020332717190387), (5, 2.377079482439926), (6, 2.467652495378928), (7, 2.744916820702403), (8, 2.5083179297597042), (9, 2.190388170055453), (10, 2.024029574861368), (11, 2.099815157116451), (12, 2.290203327171904), (13, 2.5397412199630316), (14, 2.1645101663585953), (15, 2.0369685767097967), (16, 2.319778188539741), (17, 2.157116451016636), (18, 2.1367837338262476), (19, 1.944547134935305), (20, 1.8817005545286507), (21, 2.057301293900185)

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  1.798521256931608
History (loss, distributed):
	round 1: 2.5138632162661736
	round 2: 2.622920517560074
	round 3: 2.532347504621072
	round 4: 2.9020332717190387
	round 5: 2.377079482439926
	round 6: 2.467652495378928
	round 7: 2.744916820702403
	round 8: 2.5083179297597042
	round 9: 2.190388170055453
	round 10: 2.024029574861368
	round 11: 2.099815157116451
	round 12: 2.290203327171904
	round 13: 2.5397412199630316
	round 14: 2.1645101663585953
	round 15: 2.0369685767097967
	round 16: 2.319778188539741
	round 17: 2.157116451016636
	round 18: 2.1367837338262476
	round 19: 1.944547134935305
	round 20: 1.8817005545286507
	round 21: 2.057301293900185
	round 22: 2.053604436229205
	round 23: 1.8595194085027726
	round 24: 2.0295748613678373
	round 25: 1.9704251386321627
	round 26: 2.0961182994454712
	round 27: 2.0517560073937156
	round 28: 

In [10]:
split = [0.34, 0.33, 0.33]
do_fl(split)

INFO flwr 2023-08-31 02:25:49,667 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([1860, 22])
Client 1 trainset shape torch.Size([1710, 22])
Client 2 trainset shape torch.Size([1729, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:25:58,401	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:26:06,265 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 734660198.0, 'memory': 1469320398.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:26:06,268 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:26:06,269 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:26:10,272 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:26:10,274 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:26:10,275 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:26:10,277 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:26:17,268 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:26:17,277 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  2.988909426987061


DEBUG flwr 2023-08-31 02:26:21,325 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:26:21,348 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:26:21,391 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:26:21,393 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 3)


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  1.8835489833641406


DEBUG flwr 2023-08-31 02:30:46,590 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:30:46,612 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:30:46,652 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:30:46,654 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  1.9131238447319778


DEBUG flwr 2023-08-31 02:30:51,084 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:30:51,110 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:30:51,175 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:30:51,177 | server.py:153 | FL finished in 280.8997773999872
INFO flwr 2023-08-31 02:30:51,178 | app.py:225 | app_fit: losses_distributed [(1, 2.988909426987061), (2, 4.018484288354898), (3, 2.866913123844732), (4, 2.8022181146025877), (5, 2.687615526802218), (6, 2.3715341959334566), (7, 2.1053604436229203), (8, 2.0702402957486137), (9, 2.33086876155268), (10, 2.22181146025878), (11, 2.299445471349353), (12, 2.3382624768946396), (13, 2.4861367837338264), (14, 1.9944547134935304), (15, 1.9463955637707948), (16, 2.3807763401109057), (17, 2.229205175600739), (18, 2.4861367837338264), (19, 2.210720887245841), (20, 2.0942698706099816), (21, 1.968576709796672

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  1.8151571164510167
History (loss, distributed):
	round 1: 2.988909426987061
	round 2: 4.018484288354898
	round 3: 2.866913123844732
	round 4: 2.8022181146025877
	round 5: 2.687615526802218
	round 6: 2.3715341959334566
	round 7: 2.1053604436229203
	round 8: 2.0702402957486137
	round 9: 2.33086876155268
	round 10: 2.22181146025878
	round 11: 2.299445471349353
	round 12: 2.3382624768946396
	round 13: 2.4861367837338264
	round 14: 1.9944547134935304
	round 15: 1.9463955637707948
	round 16: 2.3807763401109057
	round 17: 2.229205175600739
	round 18: 2.4861367837338264
	round 19: 2.210720887245841
	round 20: 2.0942698706099816
	round 21: 1.9685767097966729
	round 22: 2.0591497227356745
	round 23: 2.009242144177449
	round 24: 2.1182994454713495
	round 25: 2.134935304990758
	round 26: 1.9944547134935304
	round 27: 2.1737523105360443
	round 28

In [11]:
split = [0.6, 0.2, 0.2]
do_fl(split)

<class 'torch.Tensor'>

INFO flwr 2023-08-31 02:30:51,303 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)



<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([3300, 22])
Client 1 trainset shape torch.Size([1026, 22])
Client 2 trainset shape torch.Size([1062, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:30:59,799	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:31:08,085 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1443655680.0, 'object_store_memory': 721827840.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:31:08,087 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:31:08,088 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:31:11,871 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:31:11,872 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:31:11,873 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:31:11,874 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:31:17,434 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:31:17,442 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  3.944547134935305


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  2.199630314232902


DEBUG flwr 2023-08-31 02:37:10,077 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:37:10,093 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:37:10,131 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:37:10,132 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  2.2402957486136783


DEBUG flwr 2023-08-31 02:37:12,811 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:37:12,836 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:37:12,877 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:37:12,878 | server.py:153 | FL finished in 361.0044227000035
INFO flwr 2023-08-31 02:37:12,879 | app.py:225 | app_fit: losses_distributed [(1, 3.944547134935305), (2, 3.2125693160813307), (3, 4.99815157116451), (4, 2.5268022181146024), (5, 2.6950092421441774), (6, 2.4602587800369684), (7, 2.622920517560074), (8, 2.7171903881700556), (9, 2.4232902033271717), (10, 2.421441774491682), (11, 2.4177449168207024), (12, 2.933456561922366), (13, 2.1515711645101665), (14, 2.155268022181146), (15, 2.199630314232902), (16, 2.232902033271719), (17, 2.744916820702403), (18, 2.6950092421441774), (19, 2.175600739371534), (20, 2.5711645101663585), (21, 2.131238447319778

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  1.9371534195933457
History (loss, distributed):
	round 1: 3.944547134935305
	round 2: 3.2125693160813307
	round 3: 4.99815157116451
	round 4: 2.5268022181146024
	round 5: 2.6950092421441774
	round 6: 2.4602587800369684
	round 7: 2.622920517560074
	round 8: 2.7171903881700556
	round 9: 2.4232902033271717
	round 10: 2.421441774491682
	round 11: 2.4177449168207024
	round 12: 2.933456561922366
	round 13: 2.1515711645101665
	round 14: 2.155268022181146
	round 15: 2.199630314232902
	round 16: 2.232902033271719
	round 17: 2.744916820702403
	round 18: 2.6950092421441774
	round 19: 2.175600739371534
	round 20: 2.5711645101663585
	round 21: 2.1312384473197783
	round 22: 2.7911275415896486
	round 23: 2.2735674676524953
	round 24: 2.1275415896487986
	round 25: 2.2957486136783736
	round 26: 2.7060998151571165
	round 27: 2.465804066543438
	round 2

In [12]:
split = [0.8, 0.1, 0.1]
do_fl(split)

INFO flwr 2023-08-31 02:37:13,000 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([4660, 22])
Client 1 trainset shape torch.Size([464, 22])
Client 2 trainset shape torch.Size([450, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:37:21,940	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:37:30,753 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 728667340.0, 'memory': 1457334683.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:37:30,755 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:37:30,756 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:37:34,430 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:37:34,430 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:37:34,431 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:37:34,431 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:37:45,122 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:37:45,130 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  2.866913123844732


DEBUG flwr 2023-08-31 02:37:46,301 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:37:46,326 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:37:46,364 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:37:46,366 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 3)


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  2.255083179297597


DEBUG flwr 2023-08-31 02:44:17,703 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:44:17,725 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:44:17,769 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:44:17,771 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 3)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  2.048059149722736


DEBUG flwr 2023-08-31 02:44:19,009 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:44:19,034 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 3)
DEBUG flwr 2023-08-31 02:44:19,081 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:44:19,082 | server.py:153 | FL finished in 404.65107530000387
INFO flwr 2023-08-31 02:44:19,083 | app.py:225 | app_fit: losses_distributed [(1, 2.866913123844732), (2, 2.9630314232902033), (3, 2.6340110905730127), (4, 2.6857670979667283), (5, 2.66543438077634), (6, 2.900184842883549), (7, 2.6580406654343807), (8, 2.8743068391866915), (9, 2.7634011090573014), (10, 2.312384473197782), (11, 2.243992606284658), (12, 2.011090573012939), (13, 2.500924214417745), (14, 2.3419593345656193), (15, 2.5083179297597042), (16, 2.1035120147874307), (17, 2.0129390018484288), (18, 2.277264325323475), (19, 2.0462107208872458), (20, 2.7948243992606283), (21, 2.345656192236

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  2.0609981515711646
History (loss, distributed):
	round 1: 2.866913123844732
	round 2: 2.9630314232902033
	round 3: 2.6340110905730127
	round 4: 2.6857670979667283
	round 5: 2.66543438077634
	round 6: 2.900184842883549
	round 7: 2.6580406654343807
	round 8: 2.8743068391866915
	round 9: 2.7634011090573014
	round 10: 2.312384473197782
	round 11: 2.243992606284658
	round 12: 2.011090573012939
	round 13: 2.500924214417745
	round 14: 2.3419593345656193
	round 15: 2.5083179297597042
	round 16: 2.1035120147874307
	round 17: 2.0129390018484288
	round 18: 2.277264325323475
	round 19: 2.0462107208872458
	round 20: 2.7948243992606283
	round 21: 2.345656192236599
	round 22: 2.1589648798521255
	round 23: 2.3512014787430684
	round 24: 2.11460258780037
	round 25: 2.1367837338262476
	round 26: 2.3900184842883547
	round 27: 1.9815157116451017
	round 2

In [13]:
split = [0.2, 0.2, 0.2, 0.2, 0.2]
do_fl(split)

INFO flwr 2023-08-31 02:44:19,206 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([1044, 22])
Client 1 trainset shape torch.Size([884, 22])
Client 2 trainset shape torch.Size([1080, 22])
Client 3 trainset shape torch.Size([1062, 22])
Client 4 trainset shape torch.Size([1140, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:44:28,078	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:44:36,841 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 749187072.0, 'memory': 1498374144.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:44:36,845 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:44:36,847 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:44:41,455 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:44:41,456 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:44:41,456 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:44:41,457 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:44:47,340 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:44:47,348 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  3.5286506469500925


DEBUG flwr 2023-08-31 02:44:50,161 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:44:50,181 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:44:50,223 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:44:50,225 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 5)


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  2.188539741219963


DEBUG flwr 2023-08-31 02:47:50,601 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:47:50,628 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:47:50,667 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:47:50,668 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  1.8188539741219962


DEBUG flwr 2023-08-31 02:47:53,690 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:47:53,714 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:47:53,758 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:47:53,759 | server.py:153 | FL finished in 192.30223239999032
INFO flwr 2023-08-31 02:47:53,760 | app.py:225 | app_fit: losses_distributed [(1, 3.5286506469500925), (2, 3.1053604436229203), (3, 2.9685767097966727), (4, 2.741219963031423), (5, 2.9537892791127542), (6, 2.646950092421442), (7, 2.656192236598891), (8, 2.487985212569316), (9, 2.5378927911275415), (10, 2.3160813308687613), (11, 2.5434380776340113), (12, 2.8391866913123844), (13, 2.6173752310536043), (14, 2.5951940850277264), (15, 2.5489833641404807), (16, 2.4565619223659887), (17, 2.7060998151571165), (18, 2.425138632162662), (19, 2.9279112754158967), (20, 2.4861367837338264), (21, 2.45101663

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  2.022181146025878
History (loss, distributed):
	round 1: 3.5286506469500925
	round 2: 3.1053604436229203
	round 3: 2.9685767097966727
	round 4: 2.741219963031423
	round 5: 2.9537892791127542
	round 6: 2.646950092421442
	round 7: 2.656192236598891
	round 8: 2.487985212569316
	round 9: 2.5378927911275415
	round 10: 2.3160813308687613
	round 11: 2.5434380776340113
	round 12: 2.8391866913123844
	round 13: 2.6173752310536043
	round 14: 2.5951940850277264
	round 15: 2.5489833641404807
	round 16: 2.4565619223659887
	round 17: 2.7060998151571165
	round 18: 2.425138632162662
	round 19: 2.9279112754158967
	round 20: 2.4861367837338264
	round 21: 2.4510166358595193
	round 22: 2.4232902033271717
	round 23: 2.2051756007393717
	round 24: 2.3419593345656193
	round 25: 2.565619223659889
	round 26: 2.0720887245841033
	round 27: 2.0961182994454712
	ro

In [14]:
split = [0.6, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

INFO flwr 2023-08-31 02:47:53,879 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([3154, 22])
Client 1 trainset shape torch.Size([551, 22])
Client 2 trainset shape torch.Size([576, 22])
Client 3 trainset shape torch.Size([480, 22])
Client 4 trainset shape torch.Size([646, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:48:02,740	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:48:11,030 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1465855182.0, 'object_store_memory': 732927590.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:48:11,032 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:48:11,034 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:48:15,274 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:48:15,275 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:48:15,275 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:48:15,276 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:48:20,369 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:48:20,379 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  3.9593345656192236


DEBUG flwr 2023-08-31 02:48:28,501 | server.py:236 | fit_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:48:28,528 | server.py:173 | evaluate_round 2: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:48:28,582 | server.py:187 | evaluate_round 2 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:48:28,583 | server.py:222 | fit_round 3: strategy sampled 2 clients (out of 5)


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  2.057301293900185


DEBUG flwr 2023-08-31 02:52:30,577 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:52:30,599 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:52:30,644 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:52:30,645 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  2.1719038817005547


DEBUG flwr 2023-08-31 02:52:38,162 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:52:38,190 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:52:38,235 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:52:38,237 | server.py:153 | FL finished in 262.96001629999955
INFO flwr 2023-08-31 02:52:38,238 | app.py:225 | app_fit: losses_distributed [(1, 3.9593345656192236), (2, 3.053604436229205), (3, 2.866913123844732), (4, 3.234750462107209), (5, 2.6173752310536043), (6, 2.4232902033271717), (7, 3.353049907578558), (8, 2.6155268022181146), (9, 2.6284658040665434), (10, 2.377079482439926), (11, 2.33456561922366), (12, 2.1219963031423292), (13, 2.4565619223659887), (14, 2.0628465804066543), (15, 2.454713493530499), (16, 2.3826247689463957), (17, 2.4269870609981514), (18, 2.1811460258780038), (19, 2.1497227356746764), (20, 2.5083179297597042), (21, 2.57855822550

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  2.410351201478743
History (loss, distributed):
	round 1: 3.9593345656192236
	round 2: 3.053604436229205
	round 3: 2.866913123844732
	round 4: 3.234750462107209
	round 5: 2.6173752310536043
	round 6: 2.4232902033271717
	round 7: 3.353049907578558
	round 8: 2.6155268022181146
	round 9: 2.6284658040665434
	round 10: 2.377079482439926
	round 11: 2.33456561922366
	round 12: 2.1219963031423292
	round 13: 2.4565619223659887
	round 14: 2.0628465804066543
	round 15: 2.454713493530499
	round 16: 2.3826247689463957
	round 17: 2.4269870609981514
	round 18: 2.1811460258780038
	round 19: 2.1497227356746764
	round 20: 2.5083179297597042
	round 21: 2.578558225508318
	round 22: 2.6931608133086877
	round 23: 1.9741219963031422
	round 24: 2.0406654343807764
	round 25: 2.0831792975970425
	round 26: 2.1811460258780038
	round 27: 2.123844731977819
	round 

In [15]:
split = [0.8, 0.05, 0.05, 0.05, 0.05]
do_fl(split)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([4640, 22])
Client 1 trainset shape torch.Size([234, 22])
Client 2 trainset shape torch.Size([156, 22])
Client 3 trainset shape torch.Size([224, 22])


INFO flwr 2023-08-31 02:52:38,388 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


Client 4 trainset shape torch.Size([156, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:52:48,827	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:52:57,311 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 735049728.0, 'memory': 1470099456.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:52:57,313 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:52:57,315 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:53:01,446 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:53:01,447 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:53:01,447 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:53:01,448 | server.py:222 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:53:15,734 | server.py:236 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-08-31 02:53:15,748 | fedavg.

agrregate_fit results list length is  2
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  1
loss is  2.6598890942698707


...

agrregate_fit results list length is  2
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  58
loss is  2.0702402957486137


DEBUG flwr 2023-08-31 02:59:11,239 | server.py:236 | fit_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:59:11,265 | server.py:173 | evaluate_round 59: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:59:11,308 | server.py:187 | evaluate_round 59 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:59:11,309 | server.py:222 | fit_round 60: strategy sampled 2 clients (out of 5)


agrregate_fit results list length is  2
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  59
loss is  2.232902033271719


DEBUG flwr 2023-08-31 02:59:11,986 | server.py:236 | fit_round 60 received 2 results and 0 failures
DEBUG flwr 2023-08-31 02:59:12,014 | server.py:173 | evaluate_round 60: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-08-31 02:59:12,064 | server.py:187 | evaluate_round 60 received 2 results and 0 failures
INFO flwr 2023-08-31 02:59:12,066 | server.py:153 | FL finished in 370.61730250000255
INFO flwr 2023-08-31 02:59:12,068 | app.py:225 | app_fit: losses_distributed [(1, 2.6598890942698707), (2, 2.9611829944547137), (3, 2.8595194085027726), (4, 2.4158964879852127), (5, 2.8780036968576708), (6, 2.255083179297597), (7, 2.4916820702402958), (8, 2.123844731977819), (9, 2.6857670979667283), (10, 2.6247689463955637), (11, 2.4713493530499075), (12, 2.454713493530499), (13, 2.2402957486136783), (14, 2.9944547134935307), (15, 2.1645101663585953), (16, 2.578558225508318), (17, 2.1386321626617377), (18, 2.286506469500924), (19, 2.0258780036968576), (20, 2.0462107208872458), (21, 2.11275415

agrregate_fit results list length is  2
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  2
server round is  60
loss is  2.0702402957486137
History (loss, distributed):
	round 1: 2.6598890942698707
	round 2: 2.9611829944547137
	round 3: 2.8595194085027726
	round 4: 2.4158964879852127
	round 5: 2.8780036968576708
	round 6: 2.255083179297597
	round 7: 2.4916820702402958
	round 8: 2.123844731977819
	round 9: 2.6857670979667283
	round 10: 2.6247689463955637
	round 11: 2.4713493530499075
	round 12: 2.454713493530499
	round 13: 2.2402957486136783
	round 14: 2.9944547134935307
	round 15: 2.1645101663585953
	round 16: 2.578558225508318
	round 17: 2.1386321626617377
	round 18: 2.286506469500924
	round 19: 2.0258780036968576
	round 20: 2.0462107208872458
	round 21: 2.1127541589648797
	round 22: 1.9944547134935304
	round 23: 1.9149722735674677
	round 24: 2.066543438077634
	round 25: 2.229205175600739
	round 26: 2.123844731977819
	round 27: 2.421441774491682
	roun

In [16]:
split = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([510, 22])
Client 1 trainset shape torch.Size([442, 22])
Client 2 trainset shape torch.Size([595, 22])
Client 3 trainset shape torch.Size([540, 22])
Client 4 trainset shape torch.Size([527, 22])


INFO flwr 2023-08-31 02:59:12,210 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


Client 5 trainset shape torch.Size([459, 22])
Client 6 trainset shape torch.Size([468, 22])
Client 7 trainset shape torch.Size([486, 22])
Client 8 trainset shape torch.Size([496, 22])
Client 9 trainset shape torch.Size([527, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 02:59:22,164	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 02:59:30,874 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 715800576.0, 'memory': 1431601152.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 02:59:30,878 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 02:59:30,879 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 02:59:35,161 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 02:59:35,162 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 02:59:35,163 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 02:59:35,164 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 02:59:44,309 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-31 02:59:44,338 | fedavg

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  1
loss is  4.175600739371534


...

agrregate_fit results list length is  5
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  58
loss is  2.290203327171904


DEBUG flwr 2023-08-31 03:02:31,877 | server.py:236 | fit_round 59 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:02:31,923 | server.py:173 | evaluate_round 59: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:02:32,029 | server.py:187 | evaluate_round 59 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:02:32,031 | server.py:222 | fit_round 60: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  59
loss is  2.2754158964879854


DEBUG flwr 2023-08-31 03:02:34,607 | server.py:236 | fit_round 60 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:02:34,658 | server.py:173 | evaluate_round 60: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:02:34,757 | server.py:187 | evaluate_round 60 received 5 results and 0 failures
INFO flwr 2023-08-31 03:02:34,759 | server.py:153 | FL finished in 179.59548700001324
INFO flwr 2023-08-31 03:02:34,761 | app.py:225 | app_fit: losses_distributed [(1, 4.175600739371534), (2, 3.6931608133086877), (3, 3.1848428835489835), (4, 3.258780036968577), (5, 2.7597042513863217), (6, 2.6044362292051755), (7, 2.563770794824399), (8, 2.8059149722735675), (9, 2.402957486136784), (10, 2.312384473197782), (11, 2.6377079482439925), (12, 2.290203327171904), (13, 2.2717190388170057), (14, 2.4842883548983363), (15, 2.2144177449168208), (16, 2.4085027726432533), (17, 2.589648798521257), (18, 2.3031423290203326), (19, 2.188539741219963), (20, 2.2181146025878005), (21, 2.1663585951

agrregate_fit results list length is  5
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  60
loss is  2.162661737523105
History (loss, distributed):
	round 1: 4.175600739371534
	round 2: 3.6931608133086877
	round 3: 3.1848428835489835
	round 4: 3.258780036968577
	round 5: 2.7597042513863217
	round 6: 2.6044362292051755
	round 7: 2.563770794824399
	round 8: 2.8059149722735675
	round 9: 2.402957486136784
	round 10: 2.312384473197782
	round 11: 2.6377079482439925
	round 12: 2.290203327171904
	round 13: 2.2717190388170057
	round 14: 2.4842883548983363
	round 15: 2.2144177449168208
	round 16: 2.4085027726432533
	round 17: 2.589648798521257
	round 18: 2.3031423290203326
	round 19: 2.188539741219963
	round 20: 2.2181146025878005
	round 21: 2.166358595194085
	round 22: 2.3622920517560075
	round 23: 2.210720887245841
	round 24: 2.2162661737523104
	round 25: 2.319778188539741
	round 26: 2.3290203327171906
	round 27: 2.3142329020332717
	round 2

In [17]:
split = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
do_fl(split)

INFO flwr 2023-08-31 03:02:34,909 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([3300, 22])
Client 1 trainset shape torch.Size([208, 22])
Client 2 trainset shape torch.Size([224, 22])
Client 3 trainset shape torch.Size([238, 22])
Client 4 trainset shape torch.Size([240, 22])
Client 5 trainset shape torch.Size([176, 22])
Client 6 trainset shape torch.Size([225, 22])
Client 7 trainset shape torch.Size([169, 22])
Client 8 trainset shape torch.Size([192, 22])
Client 9 trainset shape torch.Size([182, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 03:02:44,303	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 03:02:53,061 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 717877248.0, 'memory': 1435754496.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 03:02:53,065 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 03:02:53,067 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 03:02:57,267 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 03:02:57,269 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 03:02:57,270 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 03:02:57,271 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:03:05,409 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-31 03:03:05,435 | fedavg

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  1
loss is  7.075785582255083


DEBUG flwr 2023-08-31 03:03:14,720 | server.py:236 | fit_round 2 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:03:14,757 | server.py:173 | evaluate_round 2: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:03:14,847 | server.py:187 | evaluate_round 2 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:03:14,849 | server.py:222 | fit_round 3: strategy sampled 5 clients (out of 10)


...

agrregate_fit results list length is  5
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  58
loss is  2.142329020332717


DEBUG flwr 2023-08-31 03:07:50,484 | server.py:236 | fit_round 59 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:07:50,526 | server.py:173 | evaluate_round 59: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:07:50,605 | server.py:187 | evaluate_round 59 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:07:50,606 | server.py:222 | fit_round 60: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  59
loss is  2.090573012939002


DEBUG flwr 2023-08-31 03:07:51,510 | server.py:236 | fit_round 60 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:07:51,550 | server.py:173 | evaluate_round 60: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:07:51,649 | server.py:187 | evaluate_round 60 received 5 results and 0 failures
INFO flwr 2023-08-31 03:07:51,651 | server.py:153 | FL finished in 294.37982329999795
INFO flwr 2023-08-31 03:07:51,653 | app.py:225 | app_fit: losses_distributed [(1, 7.075785582255083), (2, 2.9482439926062844), (3, 2.6857670979667283), (4, 2.5268022181146024), (5, 2.4473197781885396), (6, 2.497227356746765), (7, 2.43992606284658), (8, 2.1460258780036967), (9, 2.1589648798521255), (10, 2.402957486136784), (11, 2.081330868761553), (12, 2.1164510166358594), (13, 2.4621072088724585), (14, 2.1053604436229203), (15, 2.075785582255083), (16, 2.0961182994454712), (17, 2.157116451016636), (18, 2.081330868761553), (19, 2.6487985212569316), (20, 2.3086876155268024), (21, 2.41404805914

agrregate_fit results list length is  5
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  60
loss is  1.990757855822551
History (loss, distributed):
	round 1: 7.075785582255083
	round 2: 2.9482439926062844
	round 3: 2.6857670979667283
	round 4: 2.5268022181146024
	round 5: 2.4473197781885396
	round 6: 2.497227356746765
	round 7: 2.43992606284658
	round 8: 2.1460258780036967
	round 9: 2.1589648798521255
	round 10: 2.402957486136784
	round 11: 2.081330868761553
	round 12: 2.1164510166358594
	round 13: 2.4621072088724585
	round 14: 2.1053604436229203
	round 15: 2.075785582255083
	round 16: 2.0961182994454712
	round 17: 2.157116451016636
	round 18: 2.081330868761553
	round 19: 2.6487985212569316
	round 20: 2.3086876155268024
	round 21: 2.4140480591497226
	round 22: 2.6765249537892792
	round 23: 2.698706099815157
	round 24: 2.4473197781885396
	round 25: 2.155268022181146
	round 26: 2.3049907578558226
	round 27: 2.153419593345656
	round 28

In [18]:
split = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
do_fl(split)

INFO flwr 2023-08-31 03:07:51,826 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([4200, 22])
Client 1 trainset shape torch.Size([105, 22])
Client 2 trainset shape torch.Size([168, 22])
Client 3 trainset shape torch.Size([88, 22])
Client 4 trainset shape torch.Size([88, 22])
Client 5 trainset shape torch.Size([70, 22])
Client 6 trainset shape torch.Size([104, 22])
Client 7 trainset shape torch.Size([84, 22])
Client 8 trainset shape torch.Size([84, 22])
Client 9 trainset shape torch.Size([78, 22])
FL started training on cpu
Number of available GPUs: 0


2023-08-31 03:08:02,109	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-08-31 03:08:10,396 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 739758489.0, 'memory': 1479516980.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-08-31 03:08:10,399 | server.py:89 | Initializing global parameters
INFO flwr 2023-08-31 03:08:10,400 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-08-31 03:08:14,757 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-08-31 03:08:14,758 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-08-31 03:08:14,759 | server.py:104 | FL starting
DEBUG flwr 2023-08-31 03:08:14,760 | server.py:222 | fit_round 1: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:08:22,050 | server.py:236 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-31 03:08:22,078 | fedavg

agrregate_fit results list length is  5
Saving round 1 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  1
loss is  10.277264325323475


DEBUG flwr 2023-08-31 03:08:22,953 | server.py:236 | fit_round 2 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:08:23,011 | server.py:173 | evaluate_round 2: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:08:23,131 | server.py:187 | evaluate_round 2 received 5 results and 0 failures


...

agrregate_fit results list length is  5
Saving round 58 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  58
loss is  1.9722735674676526


DEBUG flwr 2023-08-31 03:13:59,093 | server.py:236 | fit_round 59 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:13:59,135 | server.py:173 | evaluate_round 59: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:13:59,222 | server.py:187 | evaluate_round 59 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:13:59,224 | server.py:222 | fit_round 60: strategy sampled 5 clients (out of 10)


agrregate_fit results list length is  5
Saving round 59 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  59
loss is  1.9242144177449167


DEBUG flwr 2023-08-31 03:14:09,442 | server.py:236 | fit_round 60 received 5 results and 0 failures
DEBUG flwr 2023-08-31 03:14:09,503 | server.py:173 | evaluate_round 60: strategy sampled 5 clients (out of 10)
DEBUG flwr 2023-08-31 03:14:09,592 | server.py:187 | evaluate_round 60 received 5 results and 0 failures
INFO flwr 2023-08-31 03:14:09,593 | server.py:153 | FL finished in 354.83305129999644
INFO flwr 2023-08-31 03:14:09,594 | app.py:225 | app_fit: losses_distributed [(1, 10.277264325323475), (2, 9.042513863216266), (3, 2.9630314232902033), (4, 2.589648798521257), (5, 2.290203327171904), (6, 3.0258780036968576), (7, 2.2809611829944547), (8, 2.288354898336414), (9, 2.1589648798521255), (10, 3.10536044362292), (11, 2.1829944547134934), (12, 2.3512014787430684), (13, 2.077634011090573), (14, 2.5083179297597042), (15, 2.358595194085028), (16, 2.0499075785582255), (17, 2.7523105360443623), (18, 2.123844731977819), (19, 2.142329020332717), (20, 2.0979667282809613), (21, 2.059149722735

agrregate_fit results list length is  5
Saving round 60 aggregated_parameters...
agrregate_evaluate results list length is  5
server round is  60
loss is  1.88909426987061
History (loss, distributed):
	round 1: 10.277264325323475
	round 2: 9.042513863216266
	round 3: 2.9630314232902033
	round 4: 2.589648798521257
	round 5: 2.290203327171904
	round 6: 3.0258780036968576
	round 7: 2.2809611829944547
	round 8: 2.288354898336414
	round 9: 2.1589648798521255
	round 10: 3.10536044362292
	round 11: 2.1829944547134934
	round 12: 2.3512014787430684
	round 13: 2.077634011090573
	round 14: 2.5083179297597042
	round 15: 2.358595194085028
	round 16: 2.0499075785582255
	round 17: 2.7523105360443623
	round 18: 2.123844731977819
	round 19: 2.142329020332717
	round 20: 2.0979667282809613
	round 21: 2.0591497227356745
	round 22: 2.0609981515711646
	round 23: 2.101663585951941
	round 24: 2.5841035120147873
	round 25: 2.3364140480591495
	round 26: 2.1940850277264325
	round 27: 1.9371534195933457
	round 28

In [19]:
split = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
         0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
do_fl(split)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([208, 22])
Client 1 trainset shape torch.Size([208, 22])
Client 2 trainset shape torch.Size([143, 22])
Client 3 trainset shape torch.Size([240, 22])
Client 4 trainset shape torch.Size([180, 22])
Client 5 trainset shape torch.Size([195, 22])
Client 6 trainset shape torch.Size([204, 22])
Client 7 trainset shape torch.Size([169, 22])


INFO flwr 2023-09-01 01:41:43,500 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


Client 8 trainset shape torch.Size([168, 22])
Client 9 trainset shape torch.Size([208, 22])
Client 10 trainset shape torch.Size([182, 22])
Client 11 trainset shape torch.Size([195, 22])
Client 12 trainset shape torch.Size([204, 22])
Client 13 trainset shape torch.Size([195, 22])
Client 14 trainset shape torch.Size([168, 22])
Client 15 trainset shape torch.Size([160, 22])
Client 16 trainset shape torch.Size([182, 22])
Client 17 trainset shape torch.Size([208, 22])
Client 18 trainset shape torch.Size([182, 22])
Client 19 trainset shape torch.Size([180, 22])
FL started training on cpu
Number of available GPUs: 0


2023-09-01 01:41:54,843	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-09-01 01:42:04,338 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 709797888.0, 'memory': 1419595776.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-01 01:42:04,342 | server.py:89 | Initializing global parameters
INFO flwr 2023-09-01 01:42:04,345 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-09-01 01:42:11,926 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-09-01 01:42:11,926 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-09-01 01:42:11,928 | server.py:104 | FL starting
DEBUG flwr 2023-09-01 01:42:11,928 | server.py:222 | fit_round 1: strategy sampled 10 clients (out of 20)
DEBUG flwr 2023-09-01 01:42:26,525 | server.py:236 | fit_round 1 received 10 results and 0 failures


agrregate_fit results list length is  10


...

agrregate_evaluate results list length is  10
server round is  59
loss is  2.192236598890943


DEBUG flwr 2023-09-01 01:45:52,859 | server.py:236 | fit_round 60 received 10 results and 0 failures
DEBUG flwr 2023-09-01 01:45:52,934 | server.py:173 | evaluate_round 60: strategy sampled 10 clients (out of 20)


agrregate_fit results list length is  10
Saving round 60 aggregated_parameters...


DEBUG flwr 2023-09-01 01:45:53,178 | server.py:187 | evaluate_round 60 received 10 results and 0 failures
INFO flwr 2023-09-01 01:45:53,180 | server.py:153 | FL finished in 221.2520050999883
INFO flwr 2023-09-01 01:45:53,182 | app.py:225 | app_fit: losses_distributed [(1, 8.519408502772643), (2, 4.43992606284658), (3, 3.5951940850277264), (4, 3.343807763401109), (5, 3.43992606284658), (6, 3.1386321626617377), (7, 2.9833641404805915), (8, 2.9038817005545288), (9, 2.922365988909427), (10, 2.966728280961183), (11, 2.7264325323475047), (12, 2.6580406654343807), (13, 2.654343807763401), (14, 2.7615526802218113), (15, 2.5711645101663585), (16, 2.5064695009242146), (17, 2.598890942698706), (18, 2.6044362292051755), (19, 2.5822550831792976), (20, 2.5286506469500925), (21, 2.5268022181146024), (22, 2.6099815157116453), (23, 2.3512014787430684), (24, 2.345656192236599), (25, 2.43992606284658), (26, 2.44547134935305), (27, 2.4417744916820703), (28, 2.4380776340110906), (29, 2.3086876155268024), (

agrregate_evaluate results list length is  10
server round is  60
loss is  2.2365988909426986
History (loss, distributed):
	round 1: 8.519408502772643
	round 2: 4.43992606284658
	round 3: 3.5951940850277264
	round 4: 3.343807763401109
	round 5: 3.43992606284658
	round 6: 3.1386321626617377
	round 7: 2.9833641404805915
	round 8: 2.9038817005545288
	round 9: 2.922365988909427
	round 10: 2.966728280961183
	round 11: 2.7264325323475047
	round 12: 2.6580406654343807
	round 13: 2.654343807763401
	round 14: 2.7615526802218113
	round 15: 2.5711645101663585
	round 16: 2.5064695009242146
	round 17: 2.598890942698706
	round 18: 2.6044362292051755
	round 19: 2.5822550831792976
	round 20: 2.5286506469500925
	round 21: 2.5268022181146024
	round 22: 2.6099815157116453
	round 23: 2.3512014787430684
	round 24: 2.345656192236599
	round 25: 2.43992606284658
	round 26: 2.44547134935305
	round 27: 2.4417744916820703
	round 28: 2.4380776340110906
	round 29: 2.3086876155268024
	round 30: 2.367837338262477
	r

In [20]:
split = [0.60, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02,
         0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
do_fl(split)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
Fold 1:
Client 0 trainset shape torch.Size([3540, 22])
Client 1 trainset shape torch.Size([77, 22])
Client 2 trainset shape torch.Size([78, 22])
Client 3 trainset shape torch.Size([78, 22])
Client 4 trainset shape torch.Size([84, 22])


INFO flwr 2023-09-01 01:45:53,444 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=60, round_timeout=None)


Client 5 trainset shape torch.Size([105, 22])
Client 6 trainset shape torch.Size([104, 22])
Client 7 trainset shape torch.Size([108, 22])
Client 8 trainset shape torch.Size([70, 22])
Client 9 trainset shape torch.Size([77, 22])
Client 10 trainset shape torch.Size([78, 22])
Client 11 trainset shape torch.Size([84, 22])
Client 12 trainset shape torch.Size([108, 22])
Client 13 trainset shape torch.Size([112, 22])
Client 14 trainset shape torch.Size([70, 22])
Client 15 trainset shape torch.Size([72, 22])
Client 16 trainset shape torch.Size([77, 22])
Client 17 trainset shape torch.Size([65, 22])
Client 18 trainset shape torch.Size([96, 22])
Client 19 trainset shape torch.Size([126, 22])
FL started training on cpu
Number of available GPUs: 0


2023-09-01 01:46:04,112	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 
INFO flwr 2023-09-01 01:46:14,288 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 1716456654.0, 'object_store_memory': 858228326.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-01 01:46:14,292 | server.py:89 | Initializing global parameters
INFO flwr 2023-09-01 01:46:14,294 | server.py:276 | Requesting initial parameters from one random client
INFO flwr 2023-09-01 01:46:18,840 | server.py:280 | Received initial parameters from one random client
INFO flwr 2023-09-01 01:46:18,841 | server.py:91 | Evaluating initial parameters
INFO flwr 2023-09-01 01:46:18,843 | server.py:104 | FL starting
DEBUG flwr 2023-09-01 01:46:18,844 | server.py:222 | fit_round 1: strategy sampled 10 clients (out of 20)
DEBUG flwr 2023-09-01 01:46:33,468 | server.py:236 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2023-09-01 01:46:33,551 | feda

agrregate_fit results list length is  10
Saving round 1 aggregated_parameters...


...

agrregate_evaluate results list length is  10
server round is  59
loss is  2.0924214417744915


DEBUG flwr 2023-09-01 01:52:44,254 | server.py:236 | fit_round 60 received 10 results and 0 failures
DEBUG flwr 2023-09-01 01:52:44,309 | server.py:173 | evaluate_round 60: strategy sampled 10 clients (out of 20)


agrregate_fit results list length is  10
Saving round 60 aggregated_parameters...


DEBUG flwr 2023-09-01 01:52:44,549 | server.py:187 | evaluate_round 60 received 10 results and 0 failures
INFO flwr 2023-09-01 01:52:44,550 | server.py:153 | FL finished in 385.70710329999565
INFO flwr 2023-09-01 01:52:44,552 | app.py:225 | app_fit: losses_distributed [(1, 10.687615526802219), (2, 10.341959334565619), (3, 5.007393715341959), (4, 2.741219963031423), (5, 2.6266173752310538), (6, 2.532347504621072), (7, 2.532347504621072), (8, 2.5175600739371533), (9, 2.8059149722735675), (10, 2.210720887245841), (11, 2.2365988909426986), (12, 2.1940850277264325), (13, 2.297597042513863), (14, 2.201478743068392), (15, 2.3068391866913123), (16, 2.3382624768946396), (17, 2.1719038817005547), (18, 2.654343807763401), (19, 2.7597042513863217), (20, 2.430683918669131), (21, 2.3364140480591495), (22, 2.2828096118299444), (23, 2.256931608133087), (24, 2.4232902033271717), (25, 2.4824399260628467), (26, 2.7911275415896486), (27, 2.101663585951941), (28, 2.066543438077634), (29, 2.0850277264325325

agrregate_evaluate results list length is  10
server round is  60
loss is  2.186691312384473
History (loss, distributed):
	round 1: 10.687615526802219
	round 2: 10.341959334565619
	round 3: 5.007393715341959
	round 4: 2.741219963031423
	round 5: 2.6266173752310538
	round 6: 2.532347504621072
	round 7: 2.532347504621072
	round 8: 2.5175600739371533
	round 9: 2.8059149722735675
	round 10: 2.210720887245841
	round 11: 2.2365988909426986
	round 12: 2.1940850277264325
	round 13: 2.297597042513863
	round 14: 2.201478743068392
	round 15: 2.3068391866913123
	round 16: 2.3382624768946396
	round 17: 2.1719038817005547
	round 18: 2.654343807763401
	round 19: 2.7597042513863217
	round 20: 2.430683918669131
	round 21: 2.3364140480591495
	round 22: 2.2828096118299444
	round 23: 2.256931608133087
	round 24: 2.4232902033271717
	round 25: 2.4824399260628467
	round 26: 2.7911275415896486
	round 27: 2.101663585951941
	round 28: 2.066543438077634
	round 29: 2.0850277264325325
	round 30: 2.22365988909427
	